# Finetuning Stable Diffusion

In [ ]:
# fix locale, to prevent future flaky errors

import locale
locale.getpreferredencoding = lambda: "UTF-8"

For this to work we need a specific use case so we can use images of it. Since I found a cool dataset that has images of Pokemon with a BLIP generated caption of it. Also who doesn't like Pokemon? So let's use this: 

Data Source: https://huggingface.co/datasets/lambdalabs/pokemon-blip-captions

Install training scripts and dependencies:

In [1]:
!git clone https://github.com/huggingface/diffusers
!pip install ./diffusers

Cloning into 'diffusers'...
remote: Enumerating objects: 29176, done.
remote: Total 29176 (delta 0), reused 0 (delta 0), pack-reused 29176
Receiving objects: 100% (29176/29176), 33.18 MiB | 27.67 MiB/s, done.
Resolving deltas: 100% (21276/21276), done.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Processing ./diffusers
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 9.8 MB/s eta 0:00:00
  Created wheel for diffusers: filename=diffusers-0.17.0.dev0-py3-none-any.whl size=1073708 sha256=0ded392bbd432435fa54d69e4555964d281f1f69bea265d80ea6e47fd9bcef9e
  Stored in directory: /tmp/pip-ephem-wheel-cache-bzix_1v3/wheels/95/c5/3b/e1b4269f8a2584de57e75f949a185b48fc4144e9a91fc9965a
Successfully built diffusers


In [ ]:
!pip install -r ./diffusers/examples/text_to_image/requirements.txt

In [ ]:
!accelerate config default

In [ ]:
!huggingface-cli login

In [ ]:
!accelerate launch --mixed_precision="fp16" ./diffusers/examples/text_to_image/train_text_to_image.py \
  --pretrained_model_name_or_path="CompVis/stable-diffusion-v-1-4" \
  --dataset_name="lambdalabs/pokemon-blip-captions" \
  --use_ema \
  --resolution=512 --center_crop --random_flip \
  --train_batch_size=1 \
  --gradient_accumulation_steps=4 \
  --gradient_checkpointing \
  --max_train_steps=15000 \
  --learning_rate=1e-05 \
  --max_grad_norm=1 \
  --lr_scheduler="constant" --lr_warmup_steps=0 \
  --output_dir="sd-pokemon-model" 

## Inference:

In [ ]:
from diffusers import StableDiffusionPipeline

model_path = "path_to_saved_model"
pipe = StableDiffusionPipeline.from_pretrained(model_path, torch_dtype=torch.float16)
pipe.to("cuda")

image = pipe(prompt="yoda").images[0]
image.save("yoda-pokemon.png")